In [ ]:
# ライブラリーの読み込み
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

In [ ]:
# データセットの読み込み
df = pd.read_csv("https://raw.githubusercontent.com/oshimalab/info_sci_2/refs/heads/main/data/Iris_2d_2classes.csv")

# irisデータセットの中身を表示
df

In [ ]:
# setosaならば0, versicolorならば1 として y を作る
y = df.iloc[:, 2].values
y = np.where(y == 'setosa', 0, 1)

# sepal length と petal length を X とする
X = df.iloc[:, [0, 1]].values

# データをプロット
plt.scatter(X[:50, 0], X[:50, 1], color='red', marker='o', label='Setosa')
plt.scatter(X[50:100, 0], X[50:100, 1], color='blue', marker='s', label='Versicolor')
plt.xlabel('Sepal length [cm]')
plt.ylabel('Petal length [cm]')
plt.legend(loc='upper left')
plt.show()

In [ ]:
# パーセプトロンのクラスを作成
class Perceptron:
    # 学習率 eta , 繰り返し回数 n_iter , 乱数シードをセットする
    def __init__(self, eta=0.1, n_iter=1000, random_state=41):
        self.eta = eta
        self.n_iter = n_iter
        self.random_state = random_state

    # パーセプトロンの出力（予測）
    def predict(self, X):
        # 入力 X から入力総和 a を求める
        a = np.dot(X, self.w) + self.b
        # シグモイド関数で予測値 y_pred を求める
        y_pred = 1.0 / (1.0 + np.exp(-a))
        return y_pred

    # 交差エントロピー誤差の計算
    def ce_loss(self, X, y):
        # 入力 X に対する予測値 y_pred を求める
        y_pred = self.predict(X)
        # 予測値 y_pred と正解 y との誤差 loss を求める
        loss = y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred)
        loss = - np.sum(loss) / len(X)
        return loss

    # 勾配の計算
    def ce_gradient(self, X, y):
        # 入力 X に対する予測値 y_pred を求める
        y_pred = self.predict(X)
        # 誤差 loss の w での微分 dldw を求める (dldwは2成分)
        dldw = ((y_pred - y) @ X) / len(X)
        # 誤差 loss の b での微分 dldb を求める
        dldb = np.sum(y_pred - y) / len(X)
        return dldw, dldb

    # 勾配法で誤差を最小化し、wとbを求める
    def fit(self, X, y):
        # パラメーターの初期化
        rng = np.random.default_rng(self.random_state)
        self.w = rng.normal(loc=0.0, scale=0.01, size=X.shape[1])
        self.b = 0.0
        # 誤差の履歴を保存する配列を準備
        self.errors = []
        # 勾配法で n_iter 回の繰り返し計算をする
        for _ in range(self.n_iter):
            # 現在の w と b から勾配を計算
            dldw, dldb = self.ce_gradient(X, y)
            # w と b を更新
            self.w += - self.eta * dldw
            self.b += - self.eta * dldb
            # 更新した w と b を用いて誤差を計算し、配列に保存
            errors = self.ce_loss(X, y)
            self.errors.append(errors)
        return self

In [ ]:
# パーセプトロンをインスタンス化
ppn = Perceptron()

# データセット X, y で学習
ppn.fit(X, y)

# 各エポックでの誤差をプロット
plt.plot(range(1, len(ppn.errors) + 1), ppn.errors, marker='o')
plt.xlabel('Epochs')
plt.ylabel('CE loss')
plt.show()

In [ ]:
# 境界線をプロットする関数
def plot_decision_boundary(X, y, model):
  xx1, xx2 = np.meshgrid(
      np.arange(X[:,0].min()-1, X[:,0].max()+1, 0.02),
      np.arange(X[:,1].min()-1, X[:,1].max()+1, 0.02)
      )
  label = model.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
  label = label.reshape(xx1.shape)
  cmap = ListedColormap(["red", "blue"])
  plt.contourf(xx1, xx2, label, alpha=0.3, cmap=cmap)
  plt.scatter(X[:50, 0], X[:50, 1], color='red', marker='o', label='Setosa')
  plt.scatter(X[50:100, 0], X[50:100, 1], color='blue', marker='s', label='Versicolor')
  plt.xlabel('Sepal length [cm]')
  plt.ylabel('Petal length [cm]')
  plt.legend(loc='upper left')
  plt.show()

# プロット
plot_decision_boundary(X, y, ppn)